# Analysis of Chris Hughes' "Fair Shot" Plan

Analysis of cash transfer plan proposed by Chris Hughes in his 2018 book, [*FAIR SHOT: Rethinking Inequality and How We Earn*](http://www.fairshotbook.com/).

Per [Felix Salmon's review](https://www.nytimes.com/2018/02/25/books/review/chris-hughes-fair-shot.html) (summarized in my [Twitter thread](https://twitter.com/MaxGhenis/status/968216600670171136)), the elements of the plan are:

1) \$6k/year paid monthly to each adult in a household with income between \$6k-\$50k in previous year, or who earned <\$50k but (a) had cared for a dependent under age six or over age 70, or (b) were enrolled in an accredited college.

2) Households earning <\$6k in past year and who don't meet dependent or college requirements get their previous year's earnings matched 100%.

3) Funded by increasing to 50% the marginal rate on income and capital gains for households earning above \$250k.

Tax-Calculator CPS data ([documentation](http://open-source-economics.github.io/Tax-Calculator/)) lacks certain data elements necessary to fully implement this, so we make simplifying assumptions.

* **No ages of child dependents.** Instead flag tax units with child/dependent-care qualifying persons as marked on [IRS Form 2441](https://www.irs.gov/pub/irs-pdf/i2441.pdf), which considers children under age 13 and disabled people (`f2441 > 0`).
* **No ages of elderly parent dependents.** Instead flag all tax units that claimed an elderly parent as a dependent (`elderly_dependent`, which has [no age threshold](https://turbotax.intuit.com/tax-tips/family/steps-to-claiming-an-elderly-parent-as-a-dependent/L34jePeT9)).
* **No college enrollment.** CPS data lacks data on lifetime learning tax expenses (`e87530`), so no substitute here. NCES [estimated](https://nces.ed.gov/fastfacts/display.asp?id=372) that 12.6 million students attended college full-time in fall 2017.
* **No knowledge of the number of adults working.** Instead determine each tax unit's eligibility and give for each adult in the tax unit. If the tax unit does not include any adults (`nu18 == XTOT`) then assume a single adult.

To simplify, this also gives money to the tax units in the same year. It's unclear whether taxdata models longitudinal variance of tax units' incomes and other situations.

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-03-01*

## Setup

### Imports

In [2]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

In [3]:
tc.__version__

'0.16.1'

In [4]:
sns.set_style('white')
DPI = 75
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi']= DPI
mpl.rcParams['figure.figsize'] = 6.4, 4.8  # Default.

In [5]:
mpl.rcParams['font.sans-serif'] = 'Roboto'
mpl.rcParams['font.family'] = 'sans-serif'

# Set title text color to dark gray (https://material.io/color) not black.
TITLE_COLOR = '#212121'
mpl.rcParams['text.color'] = TITLE_COLOR

# Axis titles and tick marks are medium gray.
AXIS_COLOR = '#757575'
mpl.rcParams['axes.labelcolor'] = AXIS_COLOR
mpl.rcParams['xtick.color'] = AXIS_COLOR
mpl.rcParams['ytick.color'] = AXIS_COLOR

# Use Seaborn's default color palette.
# https://stackoverflow.com/q/48958426/1840471 for reproducibility.
sns.set_palette(sns.color_palette())

In [57]:
# Show one decimal in tables.
pd.set_option('precision', 2)

### Utility functions

In [53]:
def weighted_sum(df, col):
    return (df[col] * df['s006']).sum()

In [7]:
def gini(x, weights=None):
    if weights is None:
        weights = np.ones_like(x)
    # Calculate mean absolute deviation in two steps, for weights.
    count = np.multiply.outer(weights, weights)
    mad = np.abs(np.subtract.outer(x, x) * count).sum() / count.sum()
    # Gini equals half the relative mean absolute deviation.
    rmad = mad / np.average(x, weights=weights)
    return 0.5 * rmad

## Model

Create `DataFrame`s for the baseline and reform scenarios. Only the tax increase is modeled as part of the taxcalc reform, since the particular form of the cash transfer isn't available. Instead it's added separately.

### Specify reform

For all tax brackets with thresholds exceeding 250k, set to 250k. Set to 50% all brackets.

In [26]:
reform = {
    2018: {
        '_II_brk4': [[157500.0, 250000.0, 157500.0, 157500.0, 250000.0]],
        '_II_brk5': [[200000.0, 250000.0, 200000.0, 200000.0, 250000.0]],
        '_II_brk6': [[250000.0, 250000.0, 250000.0, 250000.0, 250000.0]],
        '_II_rt7': [0.5]
    }
}

### Run Tax-Calculator

In [11]:
recs = tc.Records.cps_constructor()

In [165]:
base_calc = tc.Calculator(records=recs, policy=tc.Policy())
base_calc.advance_to_year(2018)
base_calc.calc_all()

base_df = base_calc.dataframe(['s006', 'XTOT', 'nu18', 'n1820', 'n21', 
                               'f2441', 'elderly_dependent',
                               'expanded_income', 'aftertax_income'])

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [166]:
pol = tc.Policy()
pol.implement_reform(reform)
reform_calc = tc.Calculator(records=recs, policy=pol)
reform_calc.advance_to_year(2018)
reform_calc.calc_all()

reform_df = reform_calc.dataframe(['aftertax_income'])

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


### Merge base and reform

In [167]:
base_df.rename(columns={'aftertax_income':'afti0'}, inplace=True)
reform_df.rename(columns={'aftertax_income':'afti1_preubi'}, inplace=True)

In [168]:
df = pd.merge(base_df, reform_df, left_index=True, right_index=True)

### Add FairShot cash amount

Pseudocode:
```
if expanded_income > 50000 then 0
elif elderly_dependent > 0 or nu18 > 0 or expanded_income > (6000 * (XTOT - nu18)) 
  then 6000 * (XTOT - nu18)
else expanded_income
```

1. Flag eligibility using expanded income.
2. Calculate maximum possible amount as `6000 * (XTOT - nu18)`.
3. Calculate eligibility for full amount based on dependents and college.
4. Calculate amount as ((1) if (2)) otherwise just expanded income.

In [169]:
df['adults'] = df['XTOT'] - df['nu18']
df['fairshot_eligible'] = df['expanded_income'] < 50000
df['fairshot_max'] = 6000 * df['adults'].clip(1)
df['fairshot_max_eligible'] = (
    (df['elderly_dependent'] > 0) |
    (df['f2441'] > 0) |
    (df['expanded_income'] >= df['fairshot_max']))
df['fairshot'] = np.where(df['fairshot_eligible'], 
                          np.where(df['fairshot_max_eligible'],
                                   df['fairshot_max'],
                                   df['expanded_income']),
                          0)
df['afti1'] = df['afti1_preubi'] + df['fairshot']

In [170]:
df[df['expanded_income'] < 6000].sample(5)

,s006,XTOT,nu18,n1820,n21,f2441,elderly_dependent,expanded_income,afti0,afti1_preubi,adults,fairshot_eligible,fairshot_max,fairshot_max_eligible,fairshot,afti1
181522,145.12,1.0,0.0,0.0,1.0,0.0,0.0,2986.13,2986.13,2986.13,1.0,True,6000.0,False,2986.13,5972.26
160524,539.40,1.0,0.0,1.0,0.0,0.0,0.0,4100.04,4100.04,4100.04,1.0,True,6000.0,False,4100.04,8200.08
455358,191.48,2.0,0.0,0.0,2.0,0.0,0.0,0.00,0.00,0.00,2.0,True,12000.0,False,0.00,0.00
29778,771.64,1.0,0.0,0.0,1.0,0.0,0.0,2513.81,2156.52,2156.52,1.0,True,6000.0,False,2513.81,4670.33
236795,2621.78,1.0,0.0,0.0,1.0,0.0,0.0,0.00,0.00,0.00,1.0,True,6000.0,False,0.00,0.00


## Analysis

### Calculate revenue from tax increases

In [135]:
'Tax increases would raise ${0:0.1f}B in revenue.'.format(
    (weighted_sum(df, 'aftertax_income_base') -
     weighted_sum(df, 'aftertax_income_reform_preubi')) / 1e9)

'Tax increases would raise $86.7B in revenue.'

### Sensitivity to inclusion criteria

How many tax units are eligible but include zero adults?

In [147]:
df.loc[df['adults'] == 0].sample(5)

,s006,XTOT,nu18,n1820,n21,f2441,elderly_dependent,expanded_income,aftertax_income_base,aftertax_income_reform_preubi,fairshot_eligible,fairshot_max,fairshot_max_eligible,fairshot,n,XTOT_minus_n,adults
29958,573.23,1.0,1.0,0.0,0.0,2.0,0.0,1382.59,1284.34,1284.34,True,6000.0,True,6000.00,1.0,0.0,0.0
405016,462.83,1.0,1.0,0.0,0.0,1.0,0.0,5112.93,6338.55,6338.55,True,6000.0,True,6000.00,1.0,0.0,0.0
104734,452.35,1.0,1.0,0.0,0.0,0.0,0.0,8532.20,7320.81,7320.81,True,6000.0,True,6000.00,1.0,0.0,0.0
54587,390.15,1.0,1.0,0.0,0.0,1.0,0.0,5917.77,7395.34,7395.34,True,6000.0,True,6000.00,1.0,0.0,0.0
220800,129.48,1.0,1.0,0.0,0.0,0.0,0.0,2739.39,2544.72,2544.72,True,6000.0,False,2739.39,1.0,0.0,0.0


In [142]:
df.loc[df['fairshot_max_eligible'] & (df['fairshot_max'] == 0)].head()

,s006,XTOT,nu18,n1820,n21,f2441,elderly_dependent,expanded_income,aftertax_income_base,aftertax_income_reform_preubi,fairshot_eligible,fairshot_max,fairshot_max_eligible,fairshot,n,XTOT_minus_n,adults


In [138]:
df.loc[(df['fairshot_max_eligible']) & (df['fairshot_max'] == 0), 'adults'].sum()

0.0

## Appendix

### Check if education and child care fields are available

Potential child care fields include:

* `e32800`: "Child/dependent-care expenses for qualifying persons from Form 2441"
* `f2441`: "number of child/dependent-care qualifying persons"
* `c07180`: "Credit for child and dependent care expenses from Form 2441"

[Form 2441 instructions](https://www.irs.gov/pub/irs-pdf/i2441.pdf) state:

> A qualifying person is:
1. A qualifying child under age 13 whom you can claim
as a dependent. If the child turned 13 during the year, the
child is a qualifying person for the part of the year he or
she was under age 13.
2. Your disabled spouse who wasn't physically or
mentally able to care for himself or herself.
3. Any disabled person who wasn't physically or
mentally able to care for himself or herself whom you can claim as a dependent or could claim as a dependent
except:
a. The disabled person had gross income of $4,050 or
more,
b. The disabled person filed a joint return, or
c. You (or your spouse if filing jointly) could be claimed
as a dependent on another taxpayer's 2017 return.

This shows the number of tax units, in millions, with nonzero values of certain fields.

In [99]:
(((recs.e32800 > 0) * recs.s006).sum() / 1e6).round()

8.0

In [100]:
(((recs.f2441 > 0) * recs.s006).sum() / 1e6).round()

34.0

In [101]:
(((recs.nu18 > 0) * recs.s006).sum() / 1e6).round()

45.0

In [102]:
(((recs.c07180 > 0) * recs.s006).sum() / 1e6).round()

0.0

### Tax units with only under-18-year-olds

This is less than the total tax units above that aren't eligible because there are no adults, because of data aging.

In [124]:
((recs.nu18 == recs.XTOT) * recs.s006).sum() / 1e6

2.3229575333338626

In [125]:
((recs.nu18 == recs.XTOT) * recs.s006 * recs.XTOT).sum() / 1e6

2.3784936633338964

% of all tax units.

In [127]:
((recs.nu18 == recs.XTOT) * recs.s006).sum() / recs.s006.sum()

0.014234177317173094